In [2]:
import pandas
from google.cloud import bigquery
from google.oauth2 import service_account
#from params.py import PROJECT_ID, TABLE_ID
import sys; sys.path.append('..')
from params import PROJECT_ID, TABLE_ID
#from collection import collection
#from data_extraction import get_coordinates
#from upload_data import send_to_bq

#collection() 


In [20]:
send_to_bq()

FileNotFoundError: [Errno 2] No such file or directory: './raw_data'

: 

In [2]:


credentials = service_account.Credentials.from_service_account_file("../bq_keys.json")
client = bigquery.Client(project=PROJECT_ID, credentials=credentials)
table = client.get_table("test_dataset.test30")


# Construct the SQL query to retrieve the table data
query = f'SELECT * FROM `{table}`'

# Submit the query and fetch the results
df = client.query(query).to_dataframe()
df

,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5,x_6,y_6,z_6,x_7,y_7,z_7,x_8,y_8,z_8,x_9,y_9,z_9,x_10,y_10,z_10,x_11,y_11,z_11,x_12,y_12,z_12,x_13,y_13,z_13,x_14,y_14,z_14,x_15,y_15,z_15,x_16,y_16,z_16,x_17,y_17,z_17,x_18,y_18,z_18,x_19,y_19,z_19,x_20,y_20,z_20,target
0,0.0,0.0,0.0,0.030617,-0.025593,-0.024471,0.052424,-0.090930,-0.033044,0.055659,-0.157378,-0.038715,0.049222,-0.206274,-0.038759,0.031604,-0.169741,-0.014330,0.026351,-0.221772,-0.037330,0.029117,-0.155124,-0.044791,0.030951,-0.138266,-0.045978,0.007571,-0.163045,-0.009283,0.004393,-0.205632,-0.033765,0.012425,-0.129860,-0.034348,0.014008,-0.128321,-0.028510,-0.015352,-0.153569,-0.008907,-0.019343,-0.184335,-0.034417,-0.009376,-0.115278,-0.024837,-0.006334,-0.114446,-0.011570,-0.039170,-0.143449,-0.010533,-0.041114,-0.168253,-0.023340,-0.031356,-0.120621,-0.014337,-0.027999,-0.109138,-0.003662,a
1,0.0,0.0,0.0,0.039712,-0.009938,-0.014068,0.082862,-0.073176,-0.016867,0.100336,-0.132777,-0.017930,0.109060,-0.181419,-0.014515,0.061054,-0.181493,-0.010170,0.088003,-0.192976,-0.025545,0.074451,-0.127280,-0.029147,0.062553,-0.127172,-0.030470,0.033998,-0.191481,-0.010995,0.063322,-0.196085,-0.029340,0.051812,-0.116627,-0.031295,0.039058,-0.124022,-0.029795,0.007564,-0.188438,-0.014084,0.035253,-0.198305,-0.036287,0.028796,-0.118876,-0.028423,0.014858,-0.121275,-0.019194,-0.019736,-0.178523,-0.017697,0.009595,-0.182768,-0.032774,0.008468,-0.129140,-0.025251,-0.005569,-0.126195,-0.016695,a
2,0.0,0.0,0.0,0.028283,-0.027583,-0.028074,0.049251,-0.094604,-0.038543,0.052000,-0.163788,-0.045716,0.044315,-0.215442,-0.047283,0.029086,-0.181012,-0.012654,0.020690,-0.223811,-0.035704,0.024830,-0.161946,-0.044303,0.028244,-0.146778,-0.046157,0.004920,-0.175239,-0.005750,-0.001370,-0.210049,-0.029545,0.008413,-0.137339,-0.030156,0.011906,-0.138638,-0.024177,-0.018055,-0.164318,-0.004162,-0.024217,-0.187287,-0.029596,-0.013244,-0.123151,-0.020404,-0.008607,-0.124421,-0.007234,-0.041484,-0.151860,-0.004945,-0.044676,-0.173983,-0.018228,-0.035034,-0.128237,-0.009689,-0.030772,-0.117302,0.001057,a
3,0.0,0.0,0.0,0.042064,-0.036703,-0.009524,0.069313,-0.113975,-0.010303,0.081982,-0.184698,-0.012622,0.077904,-0.236617,-0.010728,0.043873,-0.190953,0.004702,0.064259,-0.235749,-0.010846,0.060653,-0.165986,-0.013245,0.048666,-0.155903,-0.012194,0.021905,-0.196641,0.001521,0.042590,-0.230071,-0.016266,0.040733,-0.147912,-0.015825,0.027252,-0.150937,-0.011389,-0.000695,-0.194614,-0.005253,0.018757,-0.236576,-0.027264,0.018586,-0.155801,-0.017471,0.002872,-0.147006,-0.005684,-0.026854,-0.187424,-0.012568,-0.006809,-0.232942,-0.025460,-0.002062,-0.178175,-0.015476,-0.012370,-0.157360,-0.004711,a
4,0.0,0.0,0.0,0.035891,-0.020031,-0.017844,0.067554,-0.075301,-0.023967,0.081924,-0.129552,-0.027796,0.090698,-0.173344,-0.026866,0.047956,-0.156993,-0.014713,0.067162,-0.183935,-0.030874,0.059137,-0.123290,-0.033990,0.048083,-0.118256,-0.034389,0.021812,-0.163919,-0.013156,0.041043,-0.184841,-0.031590,0.036477,-0.111153,-0.032316,0.028582,-0.116213,-0.029897,-0.002225,-0.157490,-0.014223,0.014336,-0.172692,-0.034557,0.013980,-0.105591,-0.026624,0.005605,-0.110778,-0.017790,-0.025569,-0.146199,-0.016402,-0.009016,-0.155044,-0.029003,-0.005165,-0.111559,-0.022085,-0.013341,-0.109609,-0.014636,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,0.0,0.0,0.0,0.027759,-0.009717,-0.007931,0.048734,-0.055194,-0.011336,0.059750,-0.093314,-0.014967,0.047682,-0.111453,-0.019459,0.036554,-0.141412,-0.007818,0.041717,-0.199753,-0.016856,0.044491,-0.237249,-0.025295,0.046795,-0.270831,-0.032012,0.022770,-0.145322,-0.012156,0.026829,-0.210616,-0.020809,0.031536,-0.255648,-0.029657,0.036184,-0.295196,-0.036339,0.008129,-0.135603,-0.017685,0.010649,-0.198573,-0.026775,0.014551,-0.239687

In [3]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping


# séparation des données
X = df.drop(columns=['target'])
y = df['target']

# encoder les étiquettes de classe
encoder= LabelEncoder()
y = encoder.fit_transform(y)

# diviser les données en ensembles de formation et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [23]:
# petit earlystopping des familles
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
# définir le modèle
model = Sequential(probability=True)
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2)) 
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3)) 
model.add(Dense(len(set(y)), activation='softmax')) 

# compiler le modèle
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# former le modèle
model.fit(X_train, y_train, epochs=1000, batch_size=16, validation_split=0.30, callbacks=[early_stopping])


Epoch 1/1000
53/53 [==============================] - 0s 3ms/step - loss: 1.6149 - accuracy: 0.3993 - val_loss: 1.2898 - val_accuracy: 0.4721
Epoch 2/1000
53/53 [==============================] - 0s 1ms/step - loss: 0.9727 - accuracy: 0.6703 - val_loss: 0.7119 - val_accuracy: 0.7598
Epoch 3/1000
53/53 [==============================] - 0s 1ms/step - loss: 0.5693 - accuracy: 0.8633 - val_loss: 0.4071 - val_accuracy: 0.9413
Epoch 4/1000
53/53 [==============================] - 0s 1ms/step - loss: 0.3887 - accuracy: 0.9149 - val_loss: 0.2945 - val_accuracy: 0.9553
Epoch 5/1000
53/53 [==============================] - 0s 1ms/step - loss: 0.2820 - accuracy: 0.9365 - val_loss: 0.2502 - val_accuracy: 0.9469
Epoch 6/1000
53/53 [==============================] - 0s 1ms/step - loss: 0.2403 - accuracy: 0.9496 - val_loss: 0.1975 - val_accuracy: 0.9665
Epoch 7/1000
53/53 [==============================] - 0s 2ms/step - loss: 0.2078 - accuracy: 0.9508 - val_loss: 0.1829 - val_accuracy: 0.9721
Epoch 

In [24]:
model.evaluate(X_test, y_test)

10/10 [==============================] - 0s 1ms/step - loss: 0.1171 - accuracy: 0.9664


[0.11708107590675354, 0.9664429426193237]

model.predict()

### Test avec autres mains

In [25]:
credentials = service_account.Credentials.from_service_account_file("../bq_keys.json")
client = bigquery.Client(project=PROJECT_ID, credentials=credentials)
table = client.get_table("test_dataset.test10")


# Construct the SQL query to retrieve the table data
query = f'SELECT * FROM `{table}`'

# Submit the query and fetch the results
df_test = client.query(query).to_dataframe()
df_test

,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,...,x_18,y_18,z_18,x_19,y_19,z_19,x_20,y_20,z_20,target
0,0.0,0.0,0.0,0.068996,-0.001794,-0.026495,0.129794,-0.049328,-0.037758,0.157946,...,0.015763,-0.236859,-0.053696,0.014897,-0.160986,-0.042879,0.003309,-0.143430,-0.028293,a
1,0.0,0.0,0.0,0.084189,-0.004527,-0.035077,0.159894,-0.063521,-0.049802,0.195749,...,0.022846,-0.301951,-0.067628,0.019003,-0.205834,-0.053242,0.004331,-0.182658,-0.034498,a
2,0.0,0.0,0.0,0.081866,0.003673,-0.039060,0.166663,-0.060628,-0.056427,0.204637,...,0.011525,-0.315707,-0.082841,0.011519,-0.209255,-0.069801,-0.000003,-0.184926,-0.051607,a
3,0.0,0.0,0.0,0.072380,0.001230,-0.018210,0.133630,-0.037806,-0.024918,0.162663,...,0.032818,-0.254179,-0.044583,0.030884,-0.179450,-0.034737,0.014321,-0.144736,-0.021350,a
4,0.0,0.0,0.0,0.087307,-0.013774,-0.015136,0.145934,-0.070759,-0.021705,0.181889,...,0.044853,-0.331586,-0.057311,0.039559,-0.234893,-0.044078,0.012328,-0.194194,-0.025516,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625,0.0,0.0,0.0,0.058419,-0.030207,-0.015475,0.099631,-0.108319,-0.016353,0.097771,...,0.015726,-0.307548,-0.022291,0.024386,-0.356151,-0.029392,0.031174,-0.400676,-0.034954,b
626,0.0,0.0,0.0,0.060126,-0.017372,-0.013335,0.105420,-0.098992,-0.012475,0.108897,...,0.029067,-0.332348,-0.025400,0.040817,-0.387573,-0.033769,0.049316,-0.437535,-0.039949,b
627,0.0,0.0,0.0,0.070404,-0.006071,-0.017204,0.135407,-0.109200,-0.017203,0.149792,...,0.051218,-0.426427,-0.046215,0.069479,-0.499157,-0.057802,0.083838,-0.569521,-0.065745,b
628,0.0,0.0,0.0,0.068547,-0.012165,-0.014034,0.125481,-0.104795,-0.012689,0.135076,...,0.048240,-0.398056,-0.035419,0.063773,-0.464220,-0.046029,0.074907,-0.525973,-0.053582,b


In [32]:
X_autre = df_test.drop(columns=['target'])
y_autre = df_test['target']


model.evaluate(X_autre, y_autre)

2023-06-17 16:31:14.821109: W tensorflow/core/framework/op_kernel.cc:1805] OP_REQUIRES failed at cast_op.cc:121 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: Graph execution error:

Detected at node 'Cast_1' defined at (most recent call last):
    File "/Users/paul/.pyenv/versions/3.10.6/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/paul/.pyenv/versions/3.10.6/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/Users/paul/.pyenv/versions/3.10.6/envs/wagon-give-me-a-sign/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/paul/.pyenv/versions/3.10.6/envs/wagon-give-me-a-sign/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/Users/paul/.pyenv/versions/3.10.6/envs/wagon-give-me-a-sign/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/Users/paul/.pyenv/versions/3.10.6/envs/wagon-give-me-a-sign/lib/python3.10/si

In [27]:
from load_from_bq import load_from_bq
from data_proc import preproc

df = load_from_bq()




Not balanced
[389 301 200]
(1200, 61)


In [36]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np
from sklearn.model_selection import train_test_split

X_train , X_test, y_train, y_test = preproc(df, test_size=0.3, random_state=42)
                                     
display(X_train, y_train)                              
                                     
                                     
le = LabelEncoder()
num_classes = len(np.unique(y_train))

early_stopping = EarlyStopping(monitor='val_loss', patience=5)
n_features = X_train.shape[1] 



# Transformer les données pour qu'elles aient la bonne forme pour le LSTM

# Reshaping X_train for LSTM
n_timesteps = 30
n_samples_train = np.floor(X_train.shape[0] / n_timesteps).astype(int)
X_train = np.resize(X_train, (n_samples_train*n_timesteps, n_features))
X_train_lstm = X_train.reshape(n_samples_train, n_timesteps, n_features)

# Reshaping y_train to match X_train
y_train = np.resize(y_train, (n_samples_train*n_timesteps,))
y_train_encoded = le.fit_transform(y_train)
y_train_encoded = to_categorical(y_train_encoded, num_classes)
y_train_encoded = np.resize(y_train_encoded, (n_samples_train, num_classes))

# Do the same for X_test and y_test
n_samples_test = np.floor(X_test.shape[0] / n_timesteps).astype(int)
X_test = np.resize(X_test, (n_samples_test*n_timesteps, n_features))
X_test_lstm = X_test.reshape(n_samples_test, n_timesteps, n_features)

y_test = np.resize(y_test, (n_samples_test*n_timesteps,))
y_test_encoded = le.transform(y_test)
y_test_encoded = to_categorical(y_test_encoded, num_classes)
y_test_encoded = np.resize(y_test_encoded, (n_samples_test, num_classes))

# Continue with model creation and training...



model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps, n_features)))  # première couche LSTM
model.add(Dropout(0.2))  # régularisation par dropout
model.add(Dense(100, activation='relu'))  # une couche Dense pour le traitement après le LSTM
model.add(Dropout(0.2))  # régularisation par dropout
model.add(Dense(num_classes, activation='softmax'))  # couche de sortie - 26 pour le nombre de lettres dans l'alphabet

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'], run_eagerly=True)

X_train, X_test, y_train, y_test = train_test_split(X_train_lstm, y_train_encoded, test_size=0.2, random_state=42)

model.fit(X_train_lstm, y_train_encoded , validation_split=0.30, callbacks=[early_stopping], epochs=100)


Not balanced
[389 301 200]
(1200, 61)


,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,...,z_17,x_18,y_18,z_18,x_19,y_19,z_19,x_20,y_20,z_20
169,0.042898,-0.029042,-0.022728,0.082919,-0.099549,-0.030895,0.104474,-0.167504,-0.036971,0.107749,...,-0.015610,-0.013992,-0.216523,-0.029793,-0.007387,-0.161956,-0.019320,-0.017528,-0.151642,-0.008038
97,0.039047,-0.036933,-0.011414,0.068237,-0.109745,-0.014147,0.084549,-0.178962,-0.017134,0.076609,...,-0.013834,-0.032987,-0.235866,-0.024015,-0.021585,-0.185040,-0.013501,-0.021213,-0.159538,-0.002831
31,0.040251,-0.039287,-0.011242,0.069786,-0.110896,-0.014000,0.086180,-0.179591,-0.017391,0.077457,...,-0.012279,-0.031406,-0.244421,-0.022634,-0.020061,-0.194669,-0.012388,-0.019291,-0.167679,-0.001657
12,0.032569,0.017575,-0.003696,0.068817,-0.004343,-0.005051,0.093937,-0.025705,-0.008445,0.113761,...,-0.014285,0.062636,-0.097593,-0.026137,0.048615,-0.065057,-0.020421,0.035163,-0.074106,-0.013037
35,0.040200,-0.009149,-0.013948,0.077157,-0.071257,-0.015537,0.093177,-0.137815,-0.016252,0.095015,...,-0.012004,-0.002261,-0.235486,-0.026377,0.004294,-0.180850,-0.017632,0.000837,-0.150750,-0.006808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,0.029195,-0.016635,-0.015373,0.050096,-0.067095,-0.022519,0.051992,-0.115718,-0.028059,0.031841,...,-0.027150,-0.039759,-0.173599,-0.039065,-0.042025,-0.211872,-0.046274,-0.043191,-0.246892,-0.050782
1014,0.029558,-0.022090,-0.015085,0.050419,-0.074117,-0.021154,0.050614,-0.124967,-0.026062,0.028531,...,-0.021219,-0.027074,-0.194075,-0.030775,-0.026621,-0.231186,-0.036006,-0.026238,-0.266824,-0.039445
1092,-0.024813,-0.018772,0.001381,-0.066365,-0.030849,-0.003253,-0.100299,-0.020189,-0.008094,-0.123571,...,-0.049120,-0.100396,0.027819,-0.066554,-0.129722,0.055318,-0.071779,-0.150102,0.078985,-0.073411
1179,0.027287,-0.009614,-0.012094,0.050694,-0.052471,-0.017446,0.055791,-0.095099,-0.021654,0.037982,...,-0.024528,-0.014150,-0.171592,-0.033959,-0.010971,-0.206458,-0.039750,-0.007590,-0.238021,-0.043537


,0
169,a
97,a
31,a
12,a
35,a
...,...
1106,b
1014,b
1092,b
1179,b


Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 1.7379 - accuracy: 0.0000e+00 - val_loss: 1.6163 - val_accuracy: 1.0000
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 1.6534 - accuracy: 0.4211 - val_loss: 1.5125 - val_accuracy: 1.0000
Epoch 3/100
1/1 [==============================] - 0s 29ms/step - loss: 1.5267 - accuracy: 0.8947 - val_loss: 1.3956 - val_accuracy: 1.0000
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: 1.4101 - accuracy: 1.0000 - val_loss: 1.2470 - val_accuracy: 1.0000
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: 1.2544 - accuracy: 1.0000 - val_loss: 1.0564 - val_accuracy: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 27ms/step - loss: 1.1007 - accuracy: 1.0000 - val_loss: 0.8216 - val_accuracy: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 29ms/step - loss: 0.9157 - accuracy: 1.0000 - val_loss: 0.5593 - val_accuracy: 1.0000
Epoch 8/100
1/1 [=

In [35]:
model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 30ms/step - loss: 4.7684e-07 - accuracy: 1.0000


[4.768370445162873e-07, 1.0]